In [1]:
import glob
import json
import pandas as pd
import numpy as np
import os
from pprint import pprint as pp

In [2]:
# loading testing template
import sys
myargs = sys.argv
if '-f' in myargs:
    try: FILE_NAME = myargs[2]
    except Exception as e:
        print(e)
        FILE_NAME = 'template'
else:
    os.system("cp ./template.zip ./temp/")
    FILE_NAME = 'template'
# if testing on jupyter notebook
if FILE_NAME[0] == "/":
    os.system("cp ./template.zip ./temp/")
    FILE_NAME = 'template'

In [3]:
os.system("rm -rf ./temp/template")
os.system("unzip temp/{}.zip -d temp/{}".format(FILE_NAME, FILE_NAME))
os.system("tree")

0

In [4]:
files = sorted([intent for intent in glob.glob('temp/{}/intents/*.json'.format(FILE_NAME, FILE_NAME))])
# files.remove('template/intents/Default Fallback Intent.json')
# files.remove('template/intents/consideration-no_usersays_en.json')

In [5]:
# does not assume that all intents have usersays
# assume English only
intent_hold = "none"
intent_jsons = []
for file in files:
    intent_name = file.rpartition('.')[0]
    if not intent_name.startswith(intent_hold):
        intent_jsons.append([file])
        intent_hold = intent_name
    else:
        intent_jsons[-1].append(file)

In [6]:
intents = []
for intent_json in intent_jsons:
#     try:
        intent = {}
        intent_info_json = intent_json[0]
        usersays_jsons = intent_json[1:] # should be one only

        with open(intent_info_json, encoding="utf-8") as f:
            intent_info = json.load(f)
        
        intent["INTENT_NAME"] = intent_info["name"]
        intent["INPUT_CONTEXT"] = intent_info["contexts"]
        intent["OUTPUT_CONTEXT"] = [str(c["lifespan"]) + ", " + str(c["name"])  
                                    for c in intent_info["responses"][0]["affectedContexts"]]
        intent["RESPONSES"] = []
        
        for message in intent_info["responses"][0]["messages"]:
            if message["type"] == 0:
                if type(message["speech"]) == list:
                    intent["RESPONSES"].append(message["speech"])
                else:
                    intent["RESPONSES"].append([message["speech"]])
            elif message["type"] == 4:
                intent["RESPONSES"].append([json.dumps(message["payload"])])
            else:
                pass
#         print()
        
        intent["USER_SAYS"] = []
        for usersays_json in usersays_jsons: # should be one or none
            with open(usersays_json, encoding="utf-8") as f:
                usersays_info = json.load(f)
            for usersay in usersays_info:
                intent["USER_SAYS"].append(usersay["data"][0]["text"])
        intents.append(intent)
    #     pp(intent_info)
    #     break
#     except:
        pass

In [7]:
MAX_TOTAL_RESPONSES = max([len(intent["RESPONSES"]) for intent in intents]+[1])
# create empty arrays until they have the same number of arrays
for intent in intents:
    for _ in range(MAX_TOTAL_RESPONSES - len(intent["RESPONSES"])):
        intent["RESPONSES"].append([])
    pass

In [8]:
intents

[{'INPUT_CONTEXT': [],
  'INTENT_NAME': 'Default Fallback Intent',
  'OUTPUT_CONTEXT': [],
  'RESPONSES': [["I didn't get that. Can you say it again?",
    'I missed what you said. What was that?',
    'Sorry, could you say that again?',
    'Sorry, can you say that again?',
    'Can you say that again?',
    "Sorry, I didn't get that. Can you rephrase?",
    'Sorry, what was that?',
    'One more time?',
    'What was that?',
    'Say that one more time?',
    "I didn't get that. Can you repeat?",
    'I missed that, say that again?'],
   [],
   []],
  'USER_SAYS': []},
 {'INPUT_CONTEXT': [],
  'INTENT_NAME': 'Default Welcome Intent',
  'OUTPUT_CONTEXT': ['3, init'],
  'RESPONSES': [['Welcome to ABC law partners. \nWe can help explore whether you have a legal case against someone who has broken an agreement. \nI am not a lawyer, but I provide preliminary analysis based on the facts of your case. This service is free.\nCan you briefly explain your issue? (e.g. my customer did not pay m

In [9]:
len_INPUT_CONTEXT = max([len(intent["INPUT_CONTEXT"]) for intent in intents]+[1])
len_OUTPUT_CONTEXT = max([len(intent["OUTPUT_CONTEXT"]) for intent in intents]+[1])
len_USER_SAYS = max([len(intent["USER_SAYS"]) for intent in intents]+[1])
lens_RESPONSES = []  # find out the max number of variants of the i-th response
for i in range(MAX_TOTAL_RESPONSES):
    lens_RESPONSES.append(max([len(intent["RESPONSES"][i]) for intent in intents]+[1]))
lengths = [1, len_INPUT_CONTEXT, len_OUTPUT_CONTEXT, len_USER_SAYS, *lens_RESPONSES]

In [10]:
row_length = sum(lengths)
columns = [""]*row_length
columns[0] = "INTENT_NAME"
columns[sum(lengths[:1])] = "INPUT_CONTEXT"
columns[sum(lengths[:2])] = "OUTPUT_CONTEXT"
columns[sum(lengths[:3])] = "USER_SAYS"
for i in range(MAX_TOTAL_RESPONSES):
    columns[sum(lengths[:4+i])] = "RESPONSES"

In [11]:
row_list = []
for intent in intents:
    row = [np.nan]*row_length
    row[0] = intent["INTENT_NAME"]
    row[sum(lengths[:1]):sum(lengths[:1])+len(intent["INPUT_CONTEXT"])] = intent["INPUT_CONTEXT"]
    row[sum(lengths[:2]):sum(lengths[:2])+len(intent["OUTPUT_CONTEXT"])] = intent["OUTPUT_CONTEXT"]
    row[sum(lengths[:3]):sum(lengths[:3])+len(intent["USER_SAYS"])] = intent["USER_SAYS"]
    for i in range(MAX_TOTAL_RESPONSES):
        row[sum(lengths[:4+i]):sum(lengths[:4+i])+len(intent["RESPONSES"][i])] = intent["RESPONSES"][i]
    row_list.append(row)

In [12]:
df = pd.DataFrame(row_list, columns=columns)
df

,INTENT_NAME,INPUT_CONTEXT,,OUTPUT_CONTEXT,USER_SAYS,,,,,,...,,,,,,,,,RESPONSES,RESPONSES
0,Default Fallback Intent,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Can you say that again?,"Sorry, I didn't get that. Can you rephrase?","Sorry, what was that?",One more time?,What was that?,Say that one more time?,I didn't get that. Can you repeat?,"I missed that, say that again?",NaN,NaN
1,Default Welcome Intent,NaN,NaN,"3, init",just going to say hi,heya,hello hi,howdy,hey there,hi there,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,make chatbot,NaN,NaN,NaN,i want chatbot,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,make coffee,thirsty,NaN,"0, thirsty",i want coffee,lim kopi ai mai,kopi siu dai gam xia,NaN,NaN,NaN,...,now converting coffee into code,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,make love,rich,trouble,"0, trouble",给我一个吻,i want love,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,this is a second response bubble,"{""trap"": ""trying to break this system""}"
5,make money,NaN,NaN,"2, thirsty",i want money,bo lui ah,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,make tea,thirsty,NaN,"10, woke",i want tea,lim tehsi,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,make trouble,NaN,NaN,NaN,kua simi kua,le kua si mi,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df.to_csv("temp/{}.csv".format(FILE_NAME))

In [14]:
# convert this notebook into a python script
# and then remove the last few lines
get_ipython().system('jupyter nbconvert --to script df-to-csv.ipynb')
os.system("ex -snc '$-11,$d|x' df-to-csv.py")

[NbConvertApp] Converting notebook df-to-csv.ipynb to script
[NbConvertApp] Writing 4917 bytes to df-to-csv.py


0